#### SQLite port_lite database: orders table
#### csv files: orders, ORDERS-PLACED

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 6, 24)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name qty target active spd current change percent reason market xdate'.split()

format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market,xdate
0,23,S,JASIF,"10,000",10.20,2,C9.80,SET,2022-02-02


### Print to verify before upload file

In [3]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

#df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab = pn.widgets.Tabulator(orders)
df_tab

Tabulator(value=   trade    name  ...)

In [4]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of transactional process

### Process to set target price

#### 1) Set50 records

In [5]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET50" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,6,B,IVL,3000,46.25,1,P50%3M,SET50,2022-05-31
1,8,B,KTC,2100,55.00,0,P25%,SET50,2022-04-18
2,17,S,SCC,300,400.00,0,C398,SET50,2022-04-07
3,19,S,TISCO,1000,97.75,0,CP1S,SET50,2022-04-29
4,20,S,TTB,100000,1.25,2,CP1S,SET50,2022-04-21


In [6]:
name = 'GLOBAL'
limit = 60 # 1 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,60.00,60.00,60.00,60.00,60.00
mean,21.54,21.83,21.30,12038059.63,21.59
std,1.17,1.15,1.16,6590188.15,1.11
min,18.50,18.70,18.40,3132917.00,18.50
25%,21.00,21.30,20.80,7551738.25,21.00
50%,21.70,21.85,21.45,10208463.00,21.60
75%,22.40,22.70,22.22,14933105.25,22.40
max,23.20,23.40,23.00,34457560.00,23.40


#### 2) Set100 records

In [7]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET100" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,AP,10000,9.5,0,MinP,SET100,2022-05-10
1,3,B,BCPG,9000,10.5,1,12L,SET100,2022-02-02
2,4,B,BPP,9000,14.0,1,MinP,SET100,2022-02-02
3,21,B,DOHOME,2800,15.8,1,RD20pct,SET100,2022-02-02
4,7,B,KKP,3000,62.0,1,MinP,SET100,2022-04-28
5,11,S,BCH,3000,21.0,0,CP1S,SET100,2022-05-10


In [8]:
name = 'BCPG'
limit = 120 # 1 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,120.00,120.00,120.00,120.00,120.00
mean,12.05,12.22,11.92,10420255.88,12.08
std,0.43,0.44,0.43,9155907.25,0.42
min,10.60,11.10,10.50,1637355.00,11.10
25%,11.88,12.00,11.70,5106482.50,11.90
50%,12.00,12.10,11.90,7896586.00,12.00
75%,12.20,12.40,12.10,13374381.25,12.22
max,13.10,13.40,13.00,76370800.00,13.10


#### 3) Set records

In [9]:
# Why SIS does not print out from this SQL?
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,2,B,ASK,3000,32.00,1,MinP,SET,2022-03-03
1,14,B,JASIF,10000,9.10,2,RD10pct,SET,2022-05-20
2,22,B,MCS,5000,11.40,1,RD25pct,SET,2022-02-02
3,9,B,SSP,10000,9.00,1,6L,SET,2022-02-02
4,10,B,TMT,12000,8.40,1,RD15pct,SET,2022-04-18
5,15,S,MAKRO,1500,39.00,0,C40.40,SET,2022-03-03
6,16,S,NER,9000,7.10,0,P50%,SET,2022-04-19
7,18,S,SIS,3600,28.75,1,C28.50,SET,2022-03-10


In [10]:
name = 'MCS'
limit = 120 # 1 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,120.00,120.00,120.00,120.00,120.00
mean,13.62,13.75,13.51,2516675.32,13.66
std,1.12,1.10,1.13,2329413.21,1.11
min,11.60,11.80,11.40,259671.00,11.80
25%,12.50,12.60,12.30,1053565.50,12.50
50%,13.90,14.05,13.80,1795614.50,14.05
75%,14.60,14.70,14.50,3180099.00,14.60
max,15.00,15.20,14.90,16942701.00,15.10


In [11]:
pd.read_sql_query('SELECT * FROM orders ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,AP,10000,9.50,0,MinP,SET100,2022-05-10
1,2,B,ASK,3000,32.00,1,MinP,SET,2022-03-03
2,3,B,BCPG,9000,10.50,1,12L,SET100,2022-02-02
3,4,B,BPP,9000,14.00,1,MinP,SET100,2022-02-02
4,21,B,DOHOME,2800,15.80,1,RD20pct,SET100,2022-02-02
5,6,B,IVL,3000,46.25,1,P50%3M,SET50,2022-05-31
6,14,B,JASIF,10000,9.10,2,RD10pct,SET,2022-05-20
7,7,B,KKP,3000,62.00,1,MinP,SET100,2022-04-28
8,8,B,KTC,2100,55.00,0,P25%,SET50,2022-04-18
9,22,B,MCS,5000,11.40,1,RD25pct,SET,2022-02-02
